# Generate MCQS from a given paragraph

## TODO's

- Generate Question
- Answer that question
- Generate Distractors

### Task 1 : Generate Questions

In [1]:
# Imports
import nltk
import nltk.data
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
import pandas as pd

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
# Class initializations
nlp = spacy.load('en_core_web_sm')
stemmer = LancasterStemmer()

In [78]:
# List to hold all input sentences
sentences = []

# Dictionary to hold sentences corresponding to respective discourse markers
disc_sentences = {}

# Remaining sentences which do not have discourse markers (To be used later to generate other kinds of questions)
nondisc_sentences = []

questions_list = []
questions_answers_list = []
# List of auxiliary verbs
aux_list = ['am', 'are', 'is', 'was', 'were', 'can', 'could', 'does', 'do', 'did', 'has', 'had', 'may', 'might', 'must',
            'need', 'ought', 'shall', 'should', 'will', 'would']

# List of all discourse markers
discourse_markers = ['because', 'as a result', 'since', 'when', 'although', 'for example', 'for instance']

# Different question types possible for each discourse marker
qtype = {'because': ['Why'], 'since': ['When', 'Why'], 'when': ['When'], 'although': ['Yes/No'], 'as a result': ['Why'], 
        'for example': ['Give an example where'], 'for instance': ['Give an instance where'], 'to': ['Why']}

# The argument which forms a question
target_arg = {'because': 1, 'since': 1, 'when': 1, 'although': 1, 'as a result': 2, 'for example': 1, 'for instance': 1, 
              'to': 1}

In [79]:
# Split the paragraph into sentence
def sentensify():
    global sentences
    global questions_list
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    
    #it contains the paragraph
    
    fp = open('input.txt')
    data = fp.read()
    sentences = tokenizer.tokenize(data)
    question_answer = discourse()
    for i in range(len(question_answer)):
        questions_list.append(question_answer[i][1])
        
    

In [80]:
# Function used to generate the questions from sentences which have already been pre-processed.
def generate_question(question_part, type):

    ''' 
        question_part -> Part of input sentence which forms a question
        type-> The type of question (why, where, etc)
    '''
    # Remove full stop and make first letter lower case
    question_part = question_part[0].lower() + question_part[1:]
    if(question_part[-1] == '.' or question_part[-1] == ','):
        question_part = question_part[:-1]
        
    # Capitalizing 'i' since 'I' is recognized by parsers appropriately    
    for i in range(0, len(question_part)):
        if(question_part[i] == 'i'):
            if((i == 0 and question_part[i+1] == ' ') or (question_part[i-1] == ' ' and question_part[i+1] == ' ')):
                question_part = question_part[:i] + 'I' + question_part[i + 1: ]
                
    question = ""
    if(type == 'Give an example where' or type == 'Give an instance where'):
        question = type + " " + question_part + '?'
        return question

    aux_verb = False
    res = None
    
    # Find out if auxiliary verb already exists
    for i in range(len(aux_list)):
        if(aux_list[i] in question_part.split()):
            aux_verb = True
            pos = i
            break

    # If auxiliary verb exists
    if(aux_verb):
        
        # Tokeninze the part of the sentence from which the question has to be made
        text = nltk.word_tokenize(question_part)
        tags = nltk.pos_tag(text)
        question_part = ""
        fP = False
        
        for word, tag in tags:
            if(word in ['I', 'We', 'we']):
                question_part += 'you' + " "
                fP = True
                continue
            question_part += word + " "

        # Split across the auxiliary verb and prepend it at the start of question part
        question = question_part.split(" " + aux_list[pos])
        if(fP):
             question = ["were "] + question
        else:
            question = [aux_list[pos] + " "] + question

        # If Yes/No, no need to introduce question phrase
        if(type == 'Yes/No'):
            question += ['?']
            
        elif(type != "non_disc"):
            question = [type + " "] + question + ["?"]
            
        else:
            question = question + ["?"]
         
        question = ''.join(question)

    # If auxilary verb does ot exist, it can only be some form of verb 'do'
    else:
        aux = None
        text = nltk.word_tokenize(question_part)
        tags = nltk.pos_tag(text)
        comb = ""

        '''There can be following combinations of nouns and verbs:
            NN/NNP and VBZ  -> Does
            NNS/NNPS(plural) and VBP -> Do
            NN/NNP and VBN -> Did
            NNS/NNPS(plural) and VBN -> Did
        '''
        
        for tag in tags:
            if(comb == ""):
                if(tag[1] == 'NN' or tag[1] == 'NNP'):
                    comb = 'NN'

                elif(tag[1] == 'NNS' or tag[1] == 'NNPS'):
                    comb = 'NNS'

                elif(tag[1] == 'PRP'):
                    if tag[0] in ['He','She','It']:
                        comb = 'PRPS'
                    else:
                        comb = 'PRPP'
                        tmp = question_part.split(" ")
                        tmp = tmp[1: ]
                        if(tag[0] in ['I', 'we', 'We']):
                            question_part = 'you ' + ' '.join(tmp)
                            
            if(res == None):
                res = re.match(r"VB*", tag[1])
                if(res):
                    
                    # Stem the verb
                    question_part = question_part.replace(tag[0], stemmer.stem(tag[0]))
                res = re.match(r"VBN", tag[1])
                res = re.match(r"VBD", tag[1])

        if(comb == 'NN'):
            aux = 'does'
            
        elif(comb == 'NNS'):
            aux = 'do'
            
        elif(comb == 'PRPS'):
            aux = 'does'
            
        elif(comb == 'PRPP'):
            aux = 'do'
            
        if(res and res.group() in ['VBD', 'VBN']):
            aux = 'did'

        if(aux):
            if(type == "non_disc" or type == "Yes/No"):
                question = aux + " " + question_part + "?"

            else:
                question = type + " " + aux + " " + question_part + "?"
    if(question != ""):
        question = question[0].upper() + question[1:]
    return question

In [81]:
# This function is used to get the named entities
def get_named_entities(sent):
    doc = nlp(sent)
    named_entities = [(X.text, X.label_) for X in doc.ents]
    return named_entities

In [82]:
# This function is used to get the required wh word
def get_wh_word(entity, sent):
    wh_word = ""
    if entity[1] in ['TIME', 'DATE']:
        wh_word = 'When'
        
    elif entity[1] == ['PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']:
        wh_word = 'What'
        
    elif entity[1] in ['PERSON']:
            wh_word = 'Who'
            
    elif entity[1] in ['NORP', 'FAC' ,'ORG', 'GPE', 'LOC']:
        index = sent.find(entity[0])
        if index == 0:
            wh_word = "Who"
            
        else:
            wh_word = "Where"
            
    else:
        wh_word = "Where"
    return wh_word

In [83]:
# This function generate questions based on NER templates
def generate_one_word_questions(sent):
    
    named_entities = get_named_entities(sent)
    questions = []
    
    if not named_entities:
        return questions
    
    for entity in named_entities:
        wh_word = get_wh_word(entity, sent)
        
        if(sent[-1] == '.'):
            sent = sent[:-1]
        
        if sent.find(entity[0]) == 0:
            questions.append(sent.replace(entity[0],wh_word) + '?')
            continue
       
        question = ""
        aux_verb = False
        res = None

        for i in range(len(aux_list)):
            if(aux_list[i] in sent.split()):
                aux_verb = True
                pos = i
                break
            
        if not aux_verb:
            pos = 9
        
        text = nltk.word_tokenize(sent)
        tags = nltk.pos_tag(text)
        question_part = ""
        
        if wh_word == 'When':
            word_list = sent.split(entity[0])[0].split()
            if word_list[-1] in ['in', 'at', 'on']:
                question_part = " ".join(word_list[:-1])
            else:
                question_part = " ".join(word_list)
            
            qp_text = nltk.word_tokenize(question_part)
            qp_tags = nltk.pos_tag(qp_text)
            
            question_part = ""
            
            for i, grp in enumerate(qp_tags):
                word = grp[0]
                tag = grp[1]
                if(re.match("VB*", tag) and word not in aux_list):
                    question_part += WordNetLemmatizer().lemmatize(word,'v') + " "
                else:
                    question_part += word + " "
                
            if question_part[-1] == ' ':
                question_part = question_part[:-1]
        
        else:
            for i, grp in enumerate(tags):
                
                #Break the sentence after the first non-auxiliary verb
                word = grp[0]
                tag = grp[1]

                if(re.match("VB*", tag) and word not in aux_list):
                    question_part += word

                    if i<len(tags) and 'NN' not in tags[i+1][1] and wh_word != 'When':
                        question_part += " "+ tags[i+1][0]

                    break
                question_part += word + " "
        question = question_part.split(" "+ aux_list[pos])
        question = [aux_list[pos] + " "] + question
        question = [wh_word+ " "] + question + ["?"]
        question = ''.join(question)
        questions.append(question)
    
    return questions        

In [84]:
# Preprocessing the sentence and find if any discourses are there in it. so that questions can be made from it
def discourse():
    temp = []
    target = ""
    questions = []
    global disc_sentences
    disc_sentences = {}
    for i in range(len(sentences)):
        maxLen = 9999999
        val = -1
        for j in discourse_markers:
            tmp = len(sentences[i].split(j)[0].split(' '))  
            
            # To get valid, first discourse marker.   
            if(len(sentences[i].split(j)) > 1 and tmp >= 3 and tmp < maxLen):
                maxLen = tmp
                val = j
                
        if(val != -1):

            # To initialize a list for every new key
            if(disc_sentences.get(val, 'empty') == 'empty'):
                disc_sentences[val] = []
                
            disc_sentences[val].append(sentences[i])
            temp.append(sentences[i])


    nondisc_sentences = list(set(sentences) - set(temp))
    
    t = []
    for k, v in disc_sentences.items():
        for val in range(len(v)):
            
            # Split the sentence on discourse marker and identify the question part
            question_part = disc_sentences[k][val].split(k)[target_arg[k] - 1]
            q = generate_question(question_part, qtype[k][0])
            if(q != ""):
                questions.append([disc_sentences[k][val],q])
                
                
    for question_part in nondisc_sentences:
        s = "non_disc"
        sentence = question_part
        text = nltk.word_tokenize(question_part)
        if(text[0] == 'Yes'):
            question_part = question_part[5:]
            s = "Yes/No"
            
        elif(text[0] == 'No'):
            question_part = question_part[4:]
            s = "Yes/No"
            
        q = generate_question(question_part, s)
        if(q != ""):
            questions.append([sentence,q])
        l = generate_one_word_questions(question_part)
        questions += [[sentence,i] for i in l]
    print(len(questions))
    return questions        

In [85]:
# nltk.download('wordnet')
  

In [86]:
sentensify()

17


### Task 2 : Answer the question.

In [87]:
# Uncomment if using for the first time

# !pip install -U spacy
# !pip install -U spacy-lookups-data
# !python -m spacy download en_core_web_sm
# !conda install numpy
# !conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
#!pip install allennlp

In [88]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")


/home/vyshak/anaconda3/envs/tensornlp/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/vyshak/anaconda3/envs/tensornlp/lib/python3.7/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [89]:
questions_answers_list


[]

In [90]:
# passage = """COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019. The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually. Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing special treatment. Around 1 out of every 6 people who gets COVID-19 becomes seriously ill and develops difficulty breathing. Older people, and those with underlying medical problems like high blood pressure, heart problems or diabetes, are more likely to develop serious illness. People with fever, cough and difficulty breathing should seek medical attention."""

In [91]:
global questions_answers_list
questions_answers_list = []
f = open("input.txt")
data = f.read()
for i in questions_list:
    result = predictor.predict(passage=data,question=i)
    questions_answers_list.append([i,result['best_span_str']])

In [92]:
result = predictor.predict(passage=data,question=questions_list[0])

In [93]:
result['best_span_str']

'COVID-19'

In [95]:
questions_answers_list[1]

['Where were This new virus and disease unknown before the outbreak began in?',
 'Wuhan, China']

### Task 3 : Distractor generation

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Sequential,Model
import re

In [97]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [98]:
test = test.values

In [99]:
train = train.values
answers = {}
distractors = {}
count = 0
for x in range(train.shape[0]):
    answers[train[x][0]] = train[x][1]
    a=[]
    for y in train[x][2].split(", "):
        a.append(str(y[1:-1]))
    distractors[train[x][0]] = a
    count = count+1

In [100]:
distractors["Meals can be served"]

['outside the room at 3:00 p. m.',
 'in the dining - room at 6:00 p. m.',
 'in the dining - room from 7:30 a. m. to 9:15 p. m.']

In [101]:
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub("[^a-z0-9]+"," " , sentence)
    sentence = sentence.split()

    sentence = [s for s in sentence if((len(s)>1) or (re.match("[0-9]+",s) is not None))]
    sentence = " ".join(sentence)

    return sentence

In [102]:
# Clean all the captions
a={}
d={}
for key , dist_list in distractors.items():
    for i in range(len(dist_list)):
        dist_list[i] = clean_text(dist_list[i])
    answer=clean_text(answers[key])
    key=clean_text(key)
    a[key]=answer
    d[key]=dist_list
answers=a
distractors=d

In [103]:
distractors["meals can be served"]

['outside the room at 3 00',
 'in the dining room at 6 00',
 'in the dining room from 7 30 to 9 15']

In [104]:
with open("answers.txt","w") as f:
    f.write(str(answers))
with open("distractors.txt","w") as f:
    f.write(str(distractors))

In [105]:
vocab = set()
for key in answers.keys():
    [vocab.update(key.split())]
    [vocab.update(answers[key].split())]
    [vocab.update(sentence.split()) for sentence in distractors[key]]

    
print(len(vocab))

21459


In [106]:
total = []
for key in answers.keys():
    [total.append(i) for i in key.split()]
    [total.append(i) for i in answers[key].split()]
    [total.append(i) for des in distractors[key] for i in des.split()]

print(len(total))

718584


In [107]:
import collections
counter = collections.Counter(total)
freq_cnt = dict(counter)

sorted_freq_cnt = sorted(freq_cnt.items(),reverse = True,key = lambda x:x[1])

threshold =10
sorted_freq_cnt = [x for x in sorted_freq_cnt if x[1]>threshold]
total_words = [x[0] for x in sorted_freq_cnt]
print(len(sorted_freq_cnt))

4723


In [108]:
train_distractors = {}
for key in distractors.keys():
    train_distractors[key] = []
    for dist in distractors[key]:
        dist_to_append = "StartSeq " + dist + " EndSeq"
        train_distractors[key].append(dist_to_append)
        
print(train_distractors["meals can be served"])

['StartSeq outside the room at 3 00 EndSeq', 'StartSeq in the dining room at 6 00 EndSeq', 'StartSeq in the dining room from 7 30 to 9 15 EndSeq']


In [109]:
len(total_words)

4723

In [110]:
word_to_idx = {}
idx_to_word = {}

for i,word in enumerate(total_words):
    word_to_idx[word] = i+1
    idx_to_word[i+1] = word
    
len(word_to_idx)

4723

In [111]:
word_to_idx["StartSeq"]=4724
idx_to_word[4724] = "StartSeq"

word_to_idx["EndSeq"]=4725
idx_to_word[4725] = "EndSeq"

vocab_size = len(word_to_idx)

vocab_size= vocab_size+1

In [112]:
vocab_size

4726

In [113]:
max_len=0
for key in train_distractors.keys():
    for dist in train_distractors[key]:
        max_len = max(max_len,len(dist.split()))
print(max_len)

30


In [114]:
max_q=0
for key in train_distractors.keys():
    max_q = max(max_q,len(key.split()))
print(max_q)

48


In [115]:
max_a = 0
for key in answers.keys():
    max_a = max(max_a,len(answers[key].split()))
print(max_a)

101


In [116]:
def data_generator(train_distractors,answers,word_to_idx,max_len,batch_size):
    X1,X2,X3,y = [],[],[],[]

    n=0
    while True:
        for key,dist_list in train_distractors.items():
            n+=1

            question = key
            answer = answers[key]


            seqq = [word_to_idx[wordQ] for wordQ in question.split() if wordQ in word_to_idx]
            question= pad_sequences([seqq],maxlen=max_q,value=0,padding='post')[0]


            seqa = [word_to_idx[wordA] for wordA in answer.split() if wordA in word_to_idx]
            answer = pad_sequences([seqa],maxlen=max_a,value=0,padding='post')[0]

            for dist in dist_list:
                seq = [word_to_idx[word] for word in dist.split() if word in word_to_idx]
                for i in range(1,len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]

                    xi = pad_sequences([xi],maxlen=max_len,value = 0,padding='post')[0] 
                    yi = to_categorical([yi],num_classes = vocab_size)[0]



                    X1.append(question)
                    X2.append(answer)
                    X3.append(xi)
                    y.append(yi)
                if n==batch_size:
                    yield[[np.array(X1),np.array(X2),np.array(X3)],np.array(y)]
                    X1,X2,X3,y = [],[],[],[]
                    n=0

In [117]:
f=open("glove.6B.100d.txt",encoding="utf8")

In [118]:
embedding_index = {}
for line in f:
  values=line.split()
  word = values[0]
  embedding_index[word]=np.array(values[1:],dtype='float')

In [119]:
embedding_index['apple']

array([-0.5985   , -0.46321  ,  0.13001  , -0.019576 ,  0.4603   ,
       -0.3018   ,  0.8977   , -0.65634  ,  0.66858  , -0.49164  ,
        0.037557 , -0.050889 ,  0.6451   , -0.53882  , -0.3765   ,
       -0.04312  ,  0.51384  ,  0.17783  ,  0.28596  ,  0.92063  ,
       -0.49349  , -0.48583  ,  0.61321  ,  0.78211  ,  0.19254  ,
        0.91228  , -0.055596 , -0.12512  , -0.65688  ,  0.068557 ,
        0.55629  ,  1.611    , -0.0073642, -0.48879  ,  0.45493  ,
        0.96105  , -0.063369 ,  0.17432  ,  0.9814   , -1.3125   ,
       -0.15801  , -0.54301  , -0.13888  , -0.26146  , -0.3691   ,
        0.26844  , -0.24375  , -0.19484  ,  0.62583  , -0.7377   ,
        0.38351  , -0.75004  , -0.39053  ,  0.091498 , -0.36591  ,
       -1.4715   , -0.45228  ,  0.2256   ,  1.1412   , -0.38526  ,
       -0.06716  ,  0.57288  , -0.39191  ,  0.31302  , -0.29235  ,
       -0.96157  ,  0.15154  , -0.21659  ,  0.25103  ,  0.096967 ,
        0.2843   ,  1.4296   , -0.50565  , -0.51374  , -0.4721

In [120]:
def getEmbeddingMatrix():
    emb_dim=100
    matrix = np.zeros((vocab_size,emb_dim))
    for word,idx in word_to_idx.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            matrix[idx] = embedding_vector
    return matrix

In [121]:
embedding_matrix = getEmbeddingMatrix()
embedding_matrix.shape

(4726, 100)

In [122]:
embedding_matrix[4724]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [123]:
print(max_len)

30


In [124]:
input_dist = Input(shape = (max_len,))
input_dist1=  Embedding(input_dim=vocab_size,output_dim=100,mask_zero=True)(input_dist)
input_dist2 = Dropout(0.3)(input_dist1)
input_dist3 = LSTM(256)(input_dist2)

In [125]:
input_ques = Input(shape = (max_q,))
input_ques1=  Embedding(input_dim=vocab_size,output_dim=100,mask_zero=True)(input_ques)
input_ques2 = Dropout(0.3)(input_ques1)
input_ques3 = LSTM(256)(input_ques2)

In [126]:
input_ans = Input(shape = (max_a,))
input_ans1=  Embedding(input_dim=vocab_size,output_dim=100,mask_zero=True)(input_ans)
input_ans2 = Dropout(0.3)(input_ans1)
input_ans3 = LSTM(256)(input_ans2)

In [127]:
decoder1 = add([input_dist3,input_ques3,input_ans3])
decoder2 = Dense(512 ,activation = 'relu')(decoder1)
outputs = Dense(vocab_size,activation= 'softmax')(decoder2)

model = Model(inputs = [input_ques,input_ans,input_dist],outputs = outputs)

In [128]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 48)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 101)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 30, 100)      472600      input_4[0][0]                    
____________________________________________________________________________________________

In [129]:
model.layers[3].set_weights([embedding_matrix])
model.layers[3].trainable = False  

model.layers[4].set_weights([embedding_matrix])
model.layers[4].trainable = False  
model.layers[5].set_weights([embedding_matrix])
model.layers[5].trainable = False

In [130]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam')

In [131]:
# !mkdir model_weights

In [132]:
from keras.models import load_model
model.load_weights("./model_weights/model_19.h5")

In [65]:
model

In [ ]:
epochs=20
number_of_ques = 64
steps = len(train_distractors)//number_of_ques

for i in range(epochs):
    generator = data_generator(train_distractors,answers,word_to_idx,max_len,number_of_ques)
    model.fit_generator(generator,epochs=1,steps_per_epoch = steps,verbose = 1)
    model.save("./model_weights/model_"+str(i)+".h5")

In [133]:
answers_t = {}
count = 0
for x in range(test.shape[0]):
    answers_t[test[x][0]] = test[x][1]
    count = count+1
    
a={}
for key , answer in answers_t.items():
    answer=clean_text(answers_t[key])
    key=clean_text(key)
    a[key]=answer
answers_t=a

In [134]:
print(answers_t["what the main idea of the text"])

lack of water affects california crops


In [135]:
print(len(answers_t.keys()))

10353


In [136]:
def predict_distractors(X1,X2):
    dists = []
    for j in range(3):
        in_text = "StartSeq"
        for i in range(max_len):
            sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
            sequence = pad_sequences([sequence],maxlen=max_len,padding = 'post')[0]
            XQ = []
            XA = []
            XI = []
            XQ.append(X1)
            XA.append(X2)
            XI.append(sequence)
            y_pred = model.predict([np.array(XQ),np.array(XA),np.array(XI)])

            if(i<=1):
                y_pred=np.array(y_pred)
                y_pred = y_pred.argsort()
                y_pred=y_pred[0][:]
                y_pred=y_pred[len(y_pred)-1-j]
            else:
                y_pred=y_pred.argmax()
            word = idx_to_word[y_pred]
            in_text += (' ' + word)

            if word == 'EndSeq':
                break
        final_dists = in_text.split()[1:-1]
        final_dists = ' '.join(final_dists)
        dists.append(final_dists)
    return dists

In [140]:
questions_answers_distractors_list = []

In [141]:
def generate_MCQ(que,ans):
    global questions_answers_distractors_list
    question= clean_text(que)
    answer = clean_text(ans)


#     print(question)
#     print(answer)
    seqq = [word_to_idx[wordQ] for wordQ in question.split() if wordQ in word_to_idx]
    question= pad_sequences([seqq],maxlen=max_q,value=0,padding='post')[0]

    seqa = [word_to_idx[wordA] for wordA in answer.split() if wordA in word_to_idx]
    answer = pad_sequences([seqa],maxlen=max_a,value=0,padding='post')[0]

    #   question = question.reshape((1,question.shape[0]))
    #   answer = answer.reshape((1,answer.shape[0]))


    distractor = predict_distractors(question,answer)
    distractor = str(distractor)
    questions_answers_distractors_list.append([que,ans,distractor])


In [139]:

distractor

"['russia', 'china and china', 'province china']"

In [142]:
len(questions_answers_list)

17

In [144]:
for i in range(len(questions_answers_list)):
    que = questions_answers_list[i][0]
    ans = questions_answers_list[i][1]
    generate_MCQ(que,ans)
    


In [146]:
for i in questions_answers_distractors_list:
    print(i[0],"\n")
    print(i[1],"\n")
    print(i[2],"\n\n")

Were this new virus and disease unknown before the outbreak began in Wuhan , China , in December 2019 ? 

COVID-19 

['20', '', 'the'] 


Where were This new virus and disease unknown before the outbreak began in? 

Wuhan, China 

['russia', 'china and china', 'province china'] 


Where were This new virus and disease unknown before the outbreak began in? 

Wuhan, China 

['russia', 'china and china', 'province china'] 


When were This new virus and disease unknown before the outbreak begin in Wuhan , China ,? 

December 2019 

['20 yuan', '', 'the number of the crash'] 


May some patients have aches and pains , nasal congestion , runny nose , sore throat or diarrhea ? 

sore throat or diarrhea 

['hands', 'the type skin', 'blood'] 


Do most people (about 80%) recover from the disease without nee special treatment? 

needing 

['the lack of doctors', 'patients lack', ''] 


Where did Most people ( about 80 % ) recover from the disease without needing special? 

Wuhan, China 

['russ